In [1]:
import os
import random
import itertools
import copy
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from tqdm import trange
from PIL import Image
from collections import Counter

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = './data/processed/'
back_dir = data_dir + 'background/'
eval_dir = data_dir + 'evaluation/'

In [3]:
# ensuring reproducibility
np.random.seed(0)
random.seed(0)

In [4]:
# params
percentage_split = False
augment = False

## Train Data

The train data consists of 30 alphabets and 12 drawers.

In each alphabet folder, there is 1 subfolder for each character in the alphabet. In each character subfolder, there are 20 images of this character, 1 for each of the 20 drawers.
 
- To sample a like pair, first I randomly select an alphabet from the list of training alphabets (uniform proba). Then, I randomly select a character from the list of characters in this alphabet (uniform proba). Then, I randomly select 2 drawers from the list of training drawers (uniform proba) (note they can be the same) and select their corresponding image from the subdirectory.
- To sample a dissimilar pair, first I randomly select 2 alphabets (uniform proba) (note they can be the same), then randomly select 2 drawers (uniform proba) (note they can be the same), then uniformly sample a character drawn by each of the two drawers. If the characters are the same, I redo the above step. Else, I load their corresponding image.

In [5]:
# get list of all alphabets
background_alphabets = [os.path.join(back_dir, x) for x in next(os.walk(back_dir))[1]]
background_alphabets.sort()

# list of all drawers (1 to 20)
background_drawers = list(np.arange(1, 21))

print("There are {} alphabets.".format(len(background_alphabets)))

There are 40 alphabets.


In [6]:
# 80-20 train-valid split
if percentage_split:
    valid_size = 0.2
    num_alphabets = len(background_alphabets)

    indices = list(range(num_alphabets))
    split = int(np.floor(valid_size * num_alphabets))
    np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_alphabets = [background_alphabets[idx] for idx in train_idx]
    valid_alphabets = [background_alphabets[idx] for idx in valid_idx]

    train_alphabets.sort()
    valid_alphabets.sort()

    # from 20 drawers, randomly select 12
    train_drawers = list(np.random.choice(background_drawers, size=12, replace=False))
    remaining_drawers = [x for x in background_drawers if x not in train_drawers]

    print("There are {} train alphabets".format(len(train_alphabets)))
    print("There are {} valid alphabets".format(len(valid_alphabets)))
# 30 train, 10 valid
else:
    # from 40 alphabets, randomly select 30
    train_alphabets = list(np.random.choice(background_alphabets, size=30, replace=False))
    valid_alphabets = [x for x in background_alphabets if x not in train_alphabets]
    
    train_alphabets.sort()
    valid_alphabets.sort()

    # from 20 drawers, randomly select 12
    train_drawers = list(np.random.choice(np.arange(20), size=12, replace=False))
    remaining_drawers = [x for x in background_drawers if x not in train_drawers]

In [7]:
def check_same_alph_character(filenames):
    # same alphabet
    if filenames[0].split('/')[4] == filenames[1].split('/')[4]:
        # same character
        if filenames[0].split('/')[5] == filenames[1].split('/')[5]:
            return True
        return False
    return False

In [8]:
num_iters = int(100e3 / 2)

same_drawer = 0
selected_alphabets = []
redoing = 0
img_pairs = []
label_pairs = []
for i in trange(num_iters):
    # sample a like pair
    if i % 2 == 0:
        # uniformly select 1 alphabet
        alph = np.random.choice(train_alphabets)
        selected_alphabets.append(alph)
                
        # uniformly sample 1 character
        chars = [os.path.join(alph, x) for x in next(os.walk(alph))[1]]
        char = np.random.choice(chars)
                
        # uniformly sample 2 drawers
        ds = np.random.choice(train_drawers, size=2, replace=True)
                
        # get list of filenames to read in char dir
        filenames = [
            os.path.join(char, x) for x in next(os.walk(char))[-1] if int(
                x.split("_")[1][0:2].lstrip("0")
            ) in ds
        ]
        
        # in case I get the same drawer
        if len(filenames) == 1:
            same_drawer += 1
            filenames = filenames * 2
        
        # load pair as numpy array and store
        pair = []
        for name in filenames:
            img_arr = img2array(name, gray=True, expand=True)
            img_arr = np.transpose(img_arr, (0, 3, 1, 2))
            pair.append(img_arr)        
        img_pairs.append(np.concatenate(pair, axis=0))
        
        # store ground truth lbl
        gd_truth = np.array([1], dtype=np.int64)
        label_pairs.append(gd_truth)
        
    # sample a dissimilar pair
    else:
        redo = True
        while redo:
            # uniformly select 2 alphabets
            alph = np.random.choice(train_alphabets, size=2, replace=True)
            selected_alphabets.extend(alph)

            # uniformly sample 2 drawers
            ds = np.random.choice(train_drawers, size=2, replace=True)

            filenames = []
            for i, a in enumerate(alph):
                # uniformly sample 1 character
                chars = [os.path.join(a, x) for x in next(os.walk(a))[1]]
                char = np.random.choice(chars)

                # get list of filenames to read in char dir
                name = [
                    os.path.join(char, x) for x in next(os.walk(char))[-1] if int(
                        x.split("_")[1][0:2].lstrip("0")
                    ) == ds[i]
                ]
                filenames.append(*name)
            
            # reject (same alph, same char, same drawer) and (same alph, same char, diff drawer)
            # i.e. just check if same alph and same char
            redo = check_same_alph_character(filenames)
            if redo:
                redoing += 1

        # load pair as numpy array and store
        pair = []
        for name in filenames:
            img_arr = img2array(name, gray=True, expand=True)
            img_arr = np.transpose(img_arr, (0, 3, 1, 2))
            pair.append(img_arr)        
        img_pairs.append(np.concatenate(pair, axis=0))

         # store ground truth lbl
        gd_truth = np.array([0], dtype=np.int64)
        label_pairs.append(gd_truth)

100%|██████████| 50000/50000 [02:46<00:00, 300.36it/s]


In [9]:
print("Redid {} false pairs...".format(redoing))
print("Selected the same drawer {} times...".format(same_drawer))

Redid 29 false pairs...
Selected the same drawer 2018 times...


In [10]:
# confirm that each alphabet gets approximately equal representation
Counter(selected_alphabets)

Counter({'./data/processed/background/Angelic': 2422,
         './data/processed/background/Anglo-Saxon_Futhorc': 2531,
         './data/processed/background/Armenian': 2470,
         './data/processed/background/Asomtavruli_(Georgian)': 2571,
         './data/processed/background/Atlantean': 2467,
         './data/processed/background/Aurek-Besh': 2548,
         './data/processed/background/Balinese': 2591,
         './data/processed/background/Bengali': 2512,
         './data/processed/background/Blackfoot_(Canadian_Aboriginal_Syllabics)': 2570,
         './data/processed/background/Braille': 2520,
         './data/processed/background/Burmese_(Myanmar)': 2473,
         './data/processed/background/Cyrillic': 2566,
         './data/processed/background/Early_Aramaic': 2495,
         './data/processed/background/Futurama': 2526,
         './data/processed/background/Ge_ez': 2544,
         './data/processed/background/Grantha': 2426,
         './data/processed/background/Gujarati': 252

In [11]:
# shuffle img and labels to prevent monotone (same, different) sequence
indices = list(range(len(img_pairs)))
np.random.shuffle(indices)

img_pairs = [img_pairs[idx] for idx in indices]
label_pairs = [label_pairs[idx] for idx in indices]

In [ ]:
pickle_dump(img_pairs, data_dir + 'X_train.p')
pickle_dump(label_pairs, data_dir + 'y_train.p')

## Train Data Augmentation

We add 8 transforms for each training example pair in `img_pair`. The transformation is affine with the following constraints on its parameters:

- theta $\in$ [-10, 10] uniform (rotation)
- $\rho_x$ and $\rho_y$ $\in$ [-0.3, 0.3] uniform (shear)
- $s_x$ and $s_y$ $\in$ [0.8, 1.2] uniform (scale)
- $t_x$ and $t_y$ $\in$ [-2, 2] uniform (translation)

Each of these parameters is included with probability 0.5.

In [ ]:
arr2pil = transforms.ToPILImage()

def pil2array(im):
    x = np.asarray(im, dtype=np.float32)
    x = np.expand_dims(x, axis=-1)
    x = np.expand_dims(x, axis=0)
    x = x / 255
    return x

augmented_img_pairs = []
augmented_label_pairs = []
for idx in trange(len(img_pairs)):
    # get gd truth label
    label = label_pairs[idx]
    
    # grab img pair
    pair = img_pairs[idx]
    pair = np.transpose(pair, (0, 2, 3, 1))
    im1, im2 = np.array(pair)
    
    # convert back to [0, 255] range
    im1 *= 255
    im2 *= 255
    
    # transform to PIL image
    im1, im2 = arr2pil(im1), arr2pil(im2)
    
    # compose 8 transforms
    for i in range(8):
        # randomly select transform with proba 0.5
        rot = random.choice([0, [-10, 10]])
        shear = random.choice([None, [-0.3, 0.3]])
        scale = random.choice([None, [0.8, 1.2]])
        trans = random.choice([None, [2/150, 2/150]]) # absolute value
        
        # apply affine transformation
        aff = transforms.RandomAffine(rot, trans, scale, shear)
        aug_im1, aug_im2 = aff(im1), aff(im2)
        
        # convert to numpy array
        aug_im1 = pil2array(aug_im1)
        aug_im2 = pil2array(aug_im2)
        
        # transpose to C,H,W
        aug_im1 = np.transpose(aug_im1, (0, 3, 1, 2))
        aug_im2 = np.transpose(aug_im2, (0, 3, 1, 2))
        
        # add to list
        aug_pairs = np.concatenate([aug_im1, aug_im2], axis=0)
        augmented_img_pairs.append(aug_pairs)
        augmented_label_pairs.append(label)

## Merge Train and Augmented Train

In [ ]:
# shuffle img and labels
indices = list(range(len(augmented_img_pairs)))
np.random.shuffle(indices)

augmented_img_pairs = [augmented_img_pairs[idx] for idx in indices]
augmented_label_pairs = [augmented_label_pairs[idx] for idx in indices]

In [ ]:
train_img_pairs = img_pairs + augmented_img_pairs
train_label_pairs = label_pairs + augmented_label_pairs

print("Effective Train Size: {}".format(2 * len(train_img_pairs)))

In [ ]:
pickle_dump(train_img_pairs, data_dir + 'X_train_aug.p')
pickle_dump(train_label_pairs, data_dir + 'y_train_aug.p')

## Validation Data

The validation data consists of 10 alphabets and 4 drawers.

The authors used 2 types of validation strategies for early-stopping of the model training. One of the method consists in creating a validation one-shot scenario to test the model's ability to generalize. We pick an alphabet from among the 10 available, choose 16 characters uniformly at random and select 2 of the 4 available drawers. We then select all the 16 characters produced by the first drawer, and individually compare against all 16 characters from the second drawer, with the goal of predicting the class of the character from among all of the second drawer's characters.

This process is repeated twice for all alphabets (the second time we pick the 2 other drawers), so that there are 32 one-shot learning trials for each of the 10 validation alphabets, for a total of 320 one-shot trials.

In [13]:
# from remaining 8 drawers, select 4
valid_drawers = list(np.random.choice(remaining_drawers, size=4, replace=False))
remaining_drawers = [x for x in remaining_drawers if x not in valid_drawers]

In [14]:
num_iters = len(valid_alphabets)

# number of characters to sample in each alphabet
pop = 14

valid_img_pairs = []
valid_label_pairs = []
for iter, alph in enumerate(valid_alphabets):
    print("{}/{}".format(iter+1, num_iters))
    for j in range(2):
        # grab drawers
        ds = [valid_drawers[2*j], valid_drawers[2*j + 1]]
        
        # sample pop characters uniformly
        chars = [os.path.join(alph, x) for x in next(os.walk(alph))[1]]
        chars = np.random.choice(chars, size=pop, replace=False)
        
        # grab filenames for both drawers
        filenames = []
        for d in ds:
            for char in chars:
                names = [
                    os.path.join(char, x) for x in next(os.walk(char))[-1] if int(
                        x.split("_")[1][0:2].lstrip("0")
                    ) == d
                ]
                filenames.append(*names)
        d1 = filenames[:pop]
        d2 = filenames[pop:]
        
        
        for i, left in enumerate(d1):
            way_pairs = []
            way_labels = []
            for right in d2:
                img_names = [left, right]
                pair = []
                for name in img_names:
                    img_arr = img2array(name, gray=True, expand=True)
                    img_arr = np.transpose(img_arr, (0, 3, 1, 2))
                    pair.append(img_arr) 
                # create img and store
                pair = np.concatenate(pair, axis=0)
                way_pairs.append(pair)
            
            # create pop-way task
            way_pairs = [np.expand_dims(x, axis=0) for x in way_pairs]
            way_pairs = np.concatenate(way_pairs, axis=0)
            valid_img_pairs.append(way_pairs)
            label = np.array([i], dtype=np.int64)
            valid_label_pairs.append(label)

1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10


In [15]:
pickle_dump(valid_img_pairs, data_dir + 'X_valid.p')
pickle_dump(valid_label_pairs, data_dir + 'y_valid.p')

Writing 345758400 total bytes
Writing bytes [0, 345758400]
Done!
Writing 10769 total bytes
Writing bytes [0, 10769]
Done!


## Test Data

The test set consists in 10 alphabets and 4 drawers (just like the validation set).

In [16]:
# get list of alphabets
test_alphabets = [os.path.join(eval_dir, x) for x in next(os.walk(eval_dir))[1]]
test_alphabets.sort()

# there are 20 drawers
test_drawers = remaining_drawers

In [17]:
num_iters = len(test_alphabets)

# number of characters to sample in each alphabet
pop = 20

test_img_pairs = []
test_label_pairs = []
for iter, alph in enumerate(test_alphabets):
    print("{}/{}".format(iter+1, num_iters))
    for j in range(2):
        # sample a pair of drawers
        ds = np.random.choice(test_drawers, size=2, replace=False)
        
        # sample 20 characters uniformly
        chars = [os.path.join(alph, x) for x in next(os.walk(alph))[1]]
        chars = np.random.choice(chars, size=pop, replace=False)
        
        # grab filenames for both drawers
        filenames = []
        for d in ds:
            for char in chars:
                names = [
                    os.path.join(char, x) for x in next(os.walk(char))[-1] if int(
                        x.split("_")[1][0:2].lstrip("0")
                    ) == d
                ]
                filenames.append(*names)
        d1 = filenames[:pop]
        d2 = filenames[pop:]

        for i, left in enumerate(d1):
            way_pairs = []
            way_labels = []
            for right in d2:
                img_names = [left, right]
                pair = []
                for name in img_names:
                    img_arr = img2array(name, gray=True, expand=True)
                    img_arr = np.transpose(img_arr, (0, 3, 1, 2))
                    pair.append(img_arr) 
               # create img and store
                pair = np.concatenate(pair, axis=0)
                way_pairs.append(pair)
                
            # create pop-way task
            way_pairs = [np.expand_dims(x, axis=0) for x in way_pairs]
            way_pairs = np.concatenate(way_pairs, axis=0)
            test_img_pairs.append(way_pairs)
            label = np.array([i], dtype=np.int64)
            test_label_pairs.append(label)       

1/10
2/10
3/10
4/10
5/10
6/10
7/10
8/10
9/10
10/10


In [18]:
pickle_dump(test_img_pairs, data_dir + 'X_test.p')
pickle_dump(test_label_pairs, data_dir + 'y_test.p')

Writing 705620520 total bytes
Writing bytes [0, 705620520]
Done!
Writing 15329 total bytes
Writing bytes [0, 15329]
Done!
